## Import necessary module

In [5]:
import torch.nn as nn
import numpy as np
import torch
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

## Input and Target data

In [4]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

## Dataset and Dataloader

In [7]:
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]), tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

In [8]:
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [ ]:
for x,y in train_dl:
  print(x)
  print(y)

## Define Model

In [10]:
model = nn.Linear(3,2)

In [11]:
print(model.weight)

Parameter containing:
tensor([[-0.4481,  0.0029, -0.3184],
        [-0.2618,  0.1853,  0.3460]], requires_grad=True)


In [12]:
print(model.bias)

Parameter containing:
tensor([ 0.1893, -0.2404], requires_grad=True)


In [14]:
list(model.parameters())

[Parameter containing:
 tensor([[-0.4481,  0.0029, -0.3184],
         [-0.2618,  0.1853,  0.3460]], requires_grad=True),
 Parameter containing:
 tensor([ 0.1893, -0.2404], requires_grad=True)]

In [15]:
preds = model(inputs)
preds

tensor([[-46.0204,   7.9415],
        [-60.7123,  14.3866],
        [-56.8774,  21.8817],
        [-57.1735,  -6.1742],
        [-52.7418,  23.7048],
        [-46.4714,   7.4944],
        [-61.0336,  14.5473],
        [-57.6439,  21.9659],
        [-56.7226,  -5.7271],
        [-52.6121,  24.3126],
        [-46.3417,   8.1022],
        [-61.1633,  13.9395],
        [-56.5561,  21.7210],
        [-57.3032,  -6.7820],
        [-52.2909,  24.1519]], grad_fn=<AddmmBackward0>)

## loss function

In [16]:
loss_fn = nn.functional.mse_loss

In [18]:
print(loss_fn(model(inputs), targets))

tensor(12639.2295, grad_fn=<MseLossBackward0>)


## Define optimizer

In [19]:
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

## Train the model

In [23]:
def fit(num_epoch, model, loss_fn, opt, train_dl):
  for epoch in range(num_epoch):
    for x,y in train_dl:
      pred = model(x)
      loss = loss_fn(pred, y)
      loss.backward()
      opt.step()
      opt.zero_grad()
    if(epoch+1) % 10 == 0:
      print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epoch, loss.item()))

In [24]:
fit(100, model, loss_fn, opt, train_dl)

Epoch [10/100], Loss: 111.4514
Epoch [20/100], Loss: 46.3155
Epoch [30/100], Loss: 80.3552
Epoch [40/100], Loss: 4.3130
Epoch [50/100], Loss: 26.9972
Epoch [60/100], Loss: 19.7816
Epoch [70/100], Loss: 25.1779
Epoch [80/100], Loss: 22.9975
Epoch [90/100], Loss: 4.1379
Epoch [100/100], Loss: 26.3856


## Predictions

In [25]:
preds = model(inputs)
preds

tensor([[ 58.0141,  71.0298],
        [ 79.4576,  99.5492],
        [122.3998, 134.0287],
        [ 26.4302,  41.5275],
        [ 94.1402, 114.5308],
        [ 56.8198,  70.0148],
        [ 78.7910,  99.4231],
        [122.4369, 134.5355],
        [ 27.6245,  42.5425],
        [ 94.6678, 115.4197],
        [ 57.3474,  70.9037],
        [ 78.2633,  98.5342],
        [123.0665, 134.1548],
        [ 25.9026,  40.6386],
        [ 95.3345, 115.5458]], grad_fn=<AddmmBackward0>)

In [27]:
# compare with targets
print(loss_fn(preds,targets))

tensor(16.8659, grad_fn=<MseLossBackward0>)
